# The Devil and the Coin Flip Game

If the Devil ever challenges me to a [fiddle contest](https://en.wikipedia.org/wiki/The_Devil_Went_Down_to_Georgia), I'm going to lose. I'd have a better chance at this contest:

> *You're playing a game with the devil, with your soul at stake. You're sitting at a circular table which has 4 coins, arranged in a diamond, at the 12, 3, 6, and 9 o'clock positions. You are blindfolded, and can never see the coins or the table.*

> *Your goal is to get all 4 coins showing heads, by telling the devil the position(s) of some coins to flip. We call this a "move" on your part. The devil must faithfully perform the requested flips, but may first sneakily rotate the table any number of quarter-turns, so that the coins are in different positions. You keep making moves, and the devil keeps rotating and flipping, until all 4 coins show heads.*

> *Example: You tell the devil the 12 o'clock and 6 o'clock positions. The devil could leave the table unrotated (or could rotate it a half-turn), and then flip the two coins that you specified. Or the devil could rotate the table a quarter turn in either direction, and then flip the coins that are now in the 12 o'clock and 6 o'clock positions (which were formerly at 3 o'clock and 9 o'clock).  You won't know which of these actions the devil took.*

> *What is a shortest sequence of moves that is *guaranteed* to win, no matter what the initial state of the coins, and no matter what rotations the devil applies?*

# Analysis

The player, being blindfolded, does not know the true state of the coins. So the player should represent what is known: the *set of possible states* of the coins. We call this a *belief state*. At the start of the game, each of the four coins could be either heads or tails, so that's 2<sup>4</sup> = 16 possibilities in the belief state:

     {HHHH, HHHT, HHTH, HHTT, HTHH, HTHT, HTTH, HTTT, THHH, THHT, THTH, THTT, TTHH, TTHT, TTTH, TTTT}

The idea is that even though the player doesn't know for sure what the actual state of the coins is, nor what rotations the devil performs, the player can still manipulate the belief state towards the goal belief state:

     {HHHH}

A move updates the belief state as follows: for every coin sequence in the current belief state, rotate it in every possible way, and then flip the coins specified by the position(s) in the move. Collect all these results together to form the new belief state. Solving the game means coming up with a list of moves that ends in the belief state `{'HHHH'}`. It must be a shortest possible sequence of moves, so a [breadth-first search](https://en.wikipedia.org/wiki/Breadth-first_search) is appropriate. The search space is small (just 2<sup>16</sup> possible belief states), so run time will be fast; the only issue is specifying the domain correctly. To increase the chance of getting it right, I won't try to do anything fancy, such as noticing that some coin sequences are rotational variants of other sequences.


# Implementation Choices

Here are the main concepts, and my implementation choices:

- `Coins`: A *coin sequence* (on the table) is represented as a `str` of four characters, such as `'HTTT'`. 
- `Belief`: A *belief state* is represented as a `frozenset` of `Coins` (frozen so that it can be hashed).
- `Position`: A position is  an integer index into the coin sequence; position `0` selects the `H` in `'HTTT'`
and corresponds to the 12 o'clock position; position 1 corresponds to 3 o'clock, and so on.
- `Move`: A *move* is a set of positions to flip, such as `{0, 1}`. 
- `Strategy`: A strategy for playing the game is just a list of moves. Since there is no feedback while playing
(the player is blindfolded) there is no need for decision points in the strategy.
- `all_coins()`: A belief state consisting of the set of all possible coin sequences: `{'HHHH', 'HHHT', ...}`.
- `rotations(coins)`: returns a set of all 4 rotations of the coin sequence.
- `update(belief, move)`: an updated belief state: all the coin sequences that could result from any  rotation followed by the specified flip(s).
- `flip(coins, move)`: flips the specified positions within the coin sequence.
 (But don't flip `'HHHH'`, because the game would have already ended if this were the coin sequence.)

In [1]:
from collections import deque, Counter
from itertools   import product, combinations
import random

Coins     = ''.join   # A coin sequence; a str: 'HHHT'.
Belief    = frozenset # A set of possible coin sequences: {'HHHT', 'TTTH'}
Move      = set       # A set of positions to flip: {0, 1}
Strategy  = list      # A list of Moves: [{0, 1}, {0, 1, 2, 3}, ...]

def all_coins() -> Belief:
    "Return the belief set consisting of all possible coin sequences."
    return Belief(map(Coins, product('HT', repeat=4)))

def rotations(coins) -> [Coins]: 
    "A list of all possible rotations of a coin sequence."
    return {coins[r:] + coins[:r] for r in range(4)}

def update(belief, move) -> Belief:
    "Update belief: consider all possible rotations, then flip."
    return Belief(flip(c, move)
                  for coins in belief
                  for c in rotations(coins))

def flip(coins, move) -> Coins:
    "Flip the coins in the positions specified by the move (but leave 'HHHH' alone)."
    if coins == 'HHHH': return coins
    coins = list(coins) # Need a mutable sequence
    for i in move:
        coins[i] = ('H' if coins[i] == 'T' else 'T')
    return Coins(coins)

In [2]:
', '.join(sorted(all_coins()))

'HHHH, HHHT, HHTH, HHTT, HTHH, HTHT, HTTH, HTTT, THHH, THHT, THTH, THTT, TTHH, TTHT, TTTH, TTTT'

Let's try out these functions:

In [3]:
flip('HHHT', {0, 2})

'THTT'

In [4]:
rotations('HHHT')

{'HHHT', 'HHTH', 'HTHH', 'THHH'}

In [5]:
all_coins()

frozenset({'HHHH',
           'HHHT',
           'HHTH',
           'HHTT',
           'HTHH',
           'HTHT',
           'HTTH',
           'HTTT',
           'THHH',
           'THHT',
           'THTH',
           'THTT',
           'TTHH',
           'TTHT',
           'TTTH',
           'TTTT'})

We see there are 16 coin sequences in the `all_coins` belief state. Now if we update this belief state by flipping all 4 positions, we should get a new belief state where we have eliminated the possibility of 4 tails, leaving 15 possible coin sequences:

In [6]:
update(all_coins(), {0, 1, 2, 3})

frozenset({'HHHH',
           'HHHT',
           'HHTH',
           'HHTT',
           'HTHH',
           'HTHT',
           'HTTH',
           'HTTT',
           'THHH',
           'THHT',
           'THTH',
           'THTT',
           'TTHH',
           'TTHT',
           'TTTH'})

Everything looks good so far. One more thing: we need to find all subsets of the 4 positions; these are the possible moves:

In [7]:
def powerset(sequence): 
    "All subsets of a sequence."
    return [set(c) 
            for r in range(len(sequence) + 1)
            for c in combinations(sequence, r)]

In [8]:
powerset(range(4))

[set(),
 {0},
 {1},
 {2},
 {3},
 {0, 1},
 {0, 2},
 {0, 3},
 {1, 2},
 {1, 3},
 {2, 3},
 {0, 1, 2},
 {0, 1, 3},
 {0, 2, 3},
 {1, 2, 3},
 {0, 1, 2, 3}]

# Search for a Solution

The generic function `search` does a breadth-first search starting
from a `start` state, looking for a `goal`  state, considering possible `actions` (a collection of moves) at each turn,
and computing the `result` of each action (`result` is a function such that `result(state, action)` returns the new state that results from executing the action in the current state). It works by keeping a queue of unexplored possibilities, where each entry in the queue is a pair consisting of a *strategy* (sequence of moves) and a *state* that that strategy leads to. We also keep a set of `explored` states, so that we don't repeat ourselves.

Amazingly, 
even though we want to search in the space of *belief states*, we can still use the generic search function that is designed for regular-old-states. The search for belief states just works, as long as we properly specify the start state, the goal state, and the means of moving between states.

In [9]:
def search(start, goal, actions, result) -> Strategy:
    "Breadth-first search from start state to goal; return strategy to get to goal."
    explored = set()
    queue = deque([([], start)])
    while queue:
        (strategy, state) = queue.popleft()
        if state == goal:
            return strategy
        for action in actions:
            state2 = result(state, action)
            if state2 not in explored:
                explored.add(state2)
                queue.append((strategy + [action], state2))

The `coin_search` function calls the generic `search` function to solve our specific problem:

In [10]:
def coin_search() -> Strategy: 
    "Use `search` to solve the Coin Flip problem."
    return search(start=all_coins(), goal={'HHHH'}, 
                  actions=powerset(range(4)), result=update)

# A Solution

In [11]:
coin_search()

[{0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3}]

That's a 15-move strategy that is guaranteed to lead to a win. Stop  here if all you want is the answer to the puzzle. Or you can continue on ...

----

# Verifying the Solution

I don't have a proof, but I have some evidence that the solution is correct:
- Exploring with paper and pencil, it does appear to work. 
- A colleague did the puzzle and got the same answer. 
- Running the function `random_devil` below is consistent with it working.

The function `random_devil` takes an initial coin sequence and a sequence of moves, and plays those moves with a devil that chooses rotations randomly, returning the number of moves it takes until the player wins. Note this is dealing ith concrete, individual states of the world, like `HTHH`, not belief states.

In [12]:
def random_devil(coins, strategy) -> int or None:
    """A random devil responds to moves starting from coins; 
    return the number of moves until win, or None."""
    if coins == 'HHHH': return 0
    for (i, move) in enumerate(strategy, 1):
        coins = flip(random.choice(list(rotations(coins))), move)
        if coins == 'HHHH': 
            return i
    return None

I will let the `random_devil` play 10,000 times from each possible starting coin sequence:

In [13]:
strategy = coin_search()

Counter(random_devil(coins, strategy) 
        for coins in all_coins()
        for _ in range(10000))

Counter({0: 10000,
         1: 10000,
         2: 10067,
         3: 9933,
         4: 9862,
         5: 10014,
         6: 10162,
         7: 9962,
         8: 9951,
         9: 10108,
         10: 9947,
         11: 10044,
         12: 9841,
         13: 9986,
         14: 10127,
         15: 9996})

This says that the player won all 16,000 times. (If the player had ever lost, there would have been an entry for `None` in the Counter.) This suggests the strategy is likely to win, but doesn't prove it will always win, and doesn't say anything about the possibility of a shorter solution.
The remarkable thing, which I can't explain, is that there are very nearly exactly 1,000 results for each of the move counts from 0 to 15. Can you explain that?

# Canonical Coin Sequences

Consider the four coin sequences `{'HHHT', 'HHTH', 'HTHH', 'THHH'}`. In a sense, these are all the same: they all denote the same sequence of coins with the table rotated to different degrees. Since the devil is free to rotate the table any amount at any time, we could be justified in treating all four of these as equivalent, and collapsing them into one representative member (we could arbitrarily choose the one that comes first in alphabetical order, `'HHHT'`). I will redefine `Belief` as a function that returns a `frozenset`, just like before, but makes it a set of `canonical` coin sequences.

In [14]:
def canonical(coins): return min(rotations(coins))

def Belief(coin_collection): 
    "A set of all the coin sequences in this collection, canonicalized."
    return frozenset(canonical(coins) for coins in coin_collection)

With `Belief` redefined, the result of calling `all_coins` will be different:

In [15]:
all_coins()

frozenset({'HHHH', 'HHHT', 'HHTT', 'HTHT', 'HTTT', 'TTTT'})

The starting belief set is down from 16  to 6, namely  4 heads, 3 heads, 2 adjacent heads, 2 opposite heads, 1 head, and no heads, respectively. 

# Solutions for *N* Coins

What if there are 3 coins on the table arranged in a triangle? Or 6 coins in a hexagon? To answer that, I'll generalize the three functions that have a "4" in them, `all_coins`, `rotations` and `coin_search`.  I'll also generalize `flip`, which had `'HHHH'` in it. 

In [16]:
def all_coins(N=4) -> Belief:
    "Return the belief set consisting of all possible coin sequences."
    return Belief(map(Coins, product('HT', repeat=N)))

def rotations(coins) -> [Coins]: 
    "A list of all possible rotations of a coin sequence."
    return {coins[r:] + coins[:r] for r in range(len(coins))}

def coin_search(N=4) -> Strategy: 
    "Use the generic `search` function to solve the Coin Flip problem."
    return search(start=all_coins(N), goal={'H' * N}, 
                  actions=all_moves(N), result=update)

def flip(coins, move) -> Coins:
    "Flip the coins in the positions specified by the move (but leave all 'H' alone)."
    if 'T' not in coins: return coins
    coins = list(coins) # Need a mutable sequence
    for i in move:
        coins[i] = ('H' if coins[i] == 'T' else 'T')
    return Coins(coins)

I also need to generalize `all_moves`. To compute the set of possible moves for 4 coins, I used `powerset`, and got 16 possible moves. Now I want to know the set od canonicalized moves for any *N*. To get that, I'll look at the canonicalized set of `all_coins(N)`, and for each one pull out the positions that have an `H` in them, and flip those (the positions with a `T` should be symmetric, so we don't need them as well).

In [17]:
def all_moves(N) -> [Move]:
    "All rotationally invariant moves for a sequence of N coins."
    return [set(i for (i, coin) in enumerate(coins) if coin == 'H')
            for coins in sorted(all_coins(N))]

Let's test the new definitions:

In [18]:
assert all_moves(4) == [{0, 1, 2, 3}, {0, 1, 2}, {0, 1}, {0, 2}, {0}, set()]
assert all_coins(4) == {'HHHH', 'HHHT', 'HHTT', 'HTHT', 'HTTT', 'TTTT'}
assert all_coins(5) == {'HHHHH','HHHHT', 'HHHTT','HHTHT','HHTTT', 'HTHTT', 'HTTTT', 'TTTTT'}
assert rotations('HHHHHT') == {'HHHHHT', 'HHHHTH', 'HHHTHH', 'HHTHHH', 'HTHHHH', 'THHHHH'}
assert update({'TTTTTTT'}, {3}) == {'HTTTTTT'}
assert (update(rotations('HHHHHT'), {0}) == update({'HHHHHT'}, {1 })== update({'HHHHHT'}, {2})
        == {'HHHHHH', 'HHHHTT', 'HHHTHT', 'HHTHHT'})
'ok'

'ok'

In [19]:
all_moves(4)

[{0, 1, 2, 3}, {0, 1, 2}, {0, 1}, {0, 2}, {0}, set()]

In [20]:
 rotations('HHHHHT')

{'HHHHHT', 'HHHHTH', 'HHHTHH', 'HHTHHH', 'HTHHHH', 'THHHHH'}

With 4 coins we can flip 4, flip 3, flip 2 adjacent, flip 2 opposite, flip 1, or flip nothing.
Similarly, with 4 coins there can be 4 heads 3 heads, 2 adjacent heads, 2 opposite heads, 1 head, or no heads.
If we start with 6 coins of which one is `'T'`, and do a single flip (and it doesn't matter what position that flip is), then either the Devil flips the `'T'`, in which case we get all heads, or it can flip an `'H'` which is either adjacent to, 2 away from, or 3 away from the existing `'T'`.

How many distinct canonical coin sequences are there for *N* coins from 1 to 10?

In [21]:
{N: len(all_coins(N))
 for N in range(1, 11)}

{1: 2, 2: 3, 3: 4, 4: 6, 5: 8, 6: 14, 7: 20, 8: 36, 9: 60, 10: 108}

On the one hand this is encouraging; there are only 108 canonical coin sequences of length 10, far less than the 1024 non-canonical squences. On the other hand, it is discouraging; since we are searching over the belief states, that would be 2<sup>108</sup> ≌ 10<sup>32</sup> belief states, which is infeasible. However, we should be able to easily handle up to N=7, because 2<sup>20</sup> is only a million.

# Solutions for 1 to 7 Coins

In [22]:
{N: coin_search(N) for N in range(1, 8)}

{1: [{0}],
 2: [{0, 1}, {0}, {0, 1}],
 3: None,
 4: [{0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3},
  {0, 1},
  {0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3},
  {0, 1, 2},
  {0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3},
  {0, 1},
  {0, 1, 2, 3},
  {0, 2},
  {0, 1, 2, 3}],
 5: None,
 6: None,
 7: None}

Too bad; there are no solutions for N = 3, 5, 6, or 7.  

There are solutions for N = 1, 2, 4; they have lengths 1, 3, 15, respectively. That suggests the conjecture: 

> For every *N* that is a power of 2, there will be a shortest solution of length 2<sup>*N*</sup> - 1.

> For every *N* that is not a power of 2, there will be no solution. 

# Solution for 8 Coins

For N = 8, there are 2<sup>36</sup> = 69 billion belief states and the desired solution has 255 steps. All the computations up to now have been instantaneous, but this one should take a few minutes. Let's see:

In [23]:
%time solution8 = coin_search(8)

CPU times: user 1min 20s, sys: 295 ms, total: 1min 20s
Wall time: 1min 20s


In [24]:
len(solution8)

255

Eureka! That's evidence in favor of the conjecture. But not proof. And it leaves many questions unanswered:
- Can you show there are no solutions for *N* = 9, 10, 11, ...?
- Can you prove there are no solutions for any *N* that is not a power of 2?
- Can you find a solution of length 65,535 for *N* = 16 and verify that it works?
- Can you generate a solution for any power of 2 (without proving it is shortest)?
- Can you prove there are no shorter solutions for *N* = 16?
- Can you prove the conjecture in general?
- Can you *understand* and *explain* how the solution works, rather than just listing the moves?

# Visualizing the Solution

To aid understanding, I'll print a table showing the belief state after each move, using the  canonicalized `Belief` form, lined up neatly in columns.

In [25]:
def show(moves, N=4):
    "For each move, print the move number, move, and belief state."
    belief = all_coins(N)
    order = sorted(belief)
    show_line(0, {}, belief, order, N)
    for (i, move) in enumerate(moves, 1):
        belief = update(belief, move)
        show_line(i, move, belief, order, N)

def show_line(i, move, belief, order, N):
    "Print the move number, move, and belief state."
    ordered_belief = [(coins if coins in belief else ' ' * len(coins))
                      for coins in order]
    movestr = join((i if i in move else ' ') for i in range(N))
    print('{:3} | {:8} | {} | {}'
          .format(i, movestr, join(ordered_belief, ' '), i))
    
def join(items, sep='') -> str: return sep.join(map(str, items))

In [26]:
show(coin_search(4))

  0 |          | HHHH HHHT HHTT HTHT HTTT TTTT | 0
  1 | 0123     | HHHH HHHT HHTT HTHT HTTT      | 1
  2 | 0 2      | HHHH HHHT HHTT      HTTT TTTT | 2
  3 | 0123     | HHHH HHHT HHTT      HTTT      | 3
  4 | 01       | HHHH HHHT      HTHT HTTT TTTT | 4
  5 | 0123     | HHHH HHHT      HTHT HTTT      | 5
  6 | 0 2      | HHHH HHHT           HTTT TTTT | 6
  7 | 0123     | HHHH HHHT           HTTT      | 7
  8 | 012      | HHHH      HHTT HTHT      TTTT | 8
  9 | 0123     | HHHH      HHTT HTHT           | 9
 10 | 0 2      | HHHH      HHTT           TTTT | 10
 11 | 0123     | HHHH      HHTT                | 11
 12 | 01       | HHHH           HTHT      TTTT | 12
 13 | 0123     | HHHH           HTHT           | 13
 14 | 0 2      | HHHH                     TTTT | 14
 15 | 0123     | HHHH                          | 15


We can see that every odd-numbered move flips all four coins to eliminate the possibility of `TTTT`, flipping it to `HHHH`. We can also see that moves 2, 4, and 6 flip two coins and have the effect of eventually eliminating the two "two heads" sequences from the belief state, and then move 8 eliminates the "three heads" and "one heads" sequences, while bringing back the "two heads" possibilities. Repeating moves 2, 4, and 6 in moves 10, 12, and 14 then re-eliminates the "two heads", and move 15 gets the belief state down to `{'HHHH'}`.

You could call `show(solution8)`, but the results look bad unless you have a very wide (340 characters) screen to view it on. So I'll just show `solution8` itself:



In [27]:
solution8

[{0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 4, 5},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 2, 4, 6},
 {0, 1, 2,